In [ ]:
! pip install -U git+https://github.com/IINemo/isanlp.git@discourse

In [ ]:
SERVER0 = ''
SERVER1 = ''
SERVER2 = ''

In [17]:
from isanlp.processor_remote import ProcessorRemote
from isanlp import PipelineCommon
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd

address_morph = (SERVER0, 4333)
address_syntax = (SERVER0, 5336)
address_rst = (SERVER2, 3343)


ppl = PipelineCommon([(ProcessorRemote(address_morph[0], address_morph[1], 'default'),
                 ['text'],
                 {'tokens': 'tokens',
                  'sentences': 'sentences',
                  'postag': 'mystem_postag',
                  'lemma': 'lemma'}),
                (ProcessorRemote(address_syntax[0], address_syntax[1], '0'),
                 ['tokens', 'sentences'],
                 {'syntax_dep_tree': 'syntax_dep_tree'}),
                (ConverterMystemToUd(),
                 ['mystem_postag'],
                 {'morph': 'morph',
                  'postag': 'postag'}),
                (ProcessorRemote(address_rst[0], address_rst[1], 'default'),
                 ['text', 'tokens', 'sentences', 'postag', 'morph', 'lemma', 'syntax_dep_tree'],
                 {'rst': 'rst'})
                     ])

In [18]:
text = 'При этом отправитель письма напрямую связывает удар американского беспилотника с серией терактов в париже, которая произошла 13 ноября 2015 года.'

In [19]:
%%time

result = ppl(text)

CPU times: user 10.7 ms, sys: 3.41 ms, total: 14.1 ms
Wall time: 1.41 s


In [20]:
result['rst']

In [21]:
print(result['rst'][0])

id: 6
text: При этом отправитель письма напрямую связывает удар американского беспилотника с серией терактов в париже, которая произошла 13 ноября 2015 года.
proba: 0.632245673536659
relation: elaboration_r
nuclearity: _
left: При этом отправитель письма напрямую связывает удар американского беспилотника с серией терактов в париже
right: , которая произошла 13 ноября 2015 года.
start: 0
end: 145


In [22]:
print(result['rst'][0].left)

id: 5
text: При этом отправитель письма напрямую связывает удар американского беспилотника с серией терактов в париже
proba: 0.6674711029961333
relation: same-unit_m
nuclearity: NN
left: При этом отправитель письма напрямую связывает удар американского беспилотника
right: с серией терактов в париже
start: 0
end: 104


In [23]:
print(result['rst'][0].right)

id: 3
text: , которая произошла 13 ноября 2015 года.
proba: 1.0
relation: elementary
nuclearity: 
left: None
right: None
start: 105
end: 145


In [24]:
def extr_pairs(tree):
    pp = []
    
    if tree.left:
        pp.append([tree.left.text, tree.right.text, tree.relation])
        pp += extr_pairs(tree.left)
        pp += extr_pairs(tree.right)
    return pp

In [25]:
extr_pairs(result['rst'][0])

[['При этом отправитель письма напрямую связывает удар американского беспилотника с серией терактов в париже',
  ', которая произошла 13 ноября 2015 года.',
  'elaboration_r'],
 ['При этом отправитель письма напрямую связывает удар американского беспилотника',
  'с серией терактов в париже',
  'same-unit_m'],
 ['с серией терактов', 'в париже', 'joint_m']]